In [ ]:
import os

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (20, 16)

## Table1 data (10 sec interval)

In [ ]:
FILES = [
    "CR1000_Table1 20171128.dat",
    "CR1000_Table1 20171213.dat",
    "CR1000_Table1 20171230.dat",
    "CR1000_Table1 20180104.dat",
    "CR1000_Table1 20180125.dat",
    "CR1000_Table1 20180213.dat",
    "CR1000_Table1 20180216.dat",
    "CR1000_Table1 20180219.dat",
    "CR1000_Table1 20180221.dat",
    "CR1000_Table1 20180226.dat",
    "CR1000_Table1 20180301.dat",
    "CR1000_Table1 20180305.dat",
    "CR1000_Table1 20180307.dat",
    "CR1000_Table1 20180311.dat",
    "CR1000_Table1 20180314.dat",
    "CR1000_Table1 20180319.dat",
    "CR1000_Table1 20180321.dat",
    "CR1000_Table1 20180326.dat",
    "CR1000_Table1 20180328.dat",
    "CR1000_Table1 20180401.dat",
    "CR1000_Table1 20180404.dat",
    "CR1000_Table1 20180405.dat",
    "CR1000_Table1 20180407.dat",
    "CR1000_Table1 20180410.dat",
    "CR1000_Table1 20180412.dat",
    "CR1000_Table1 20180414.dat",
    "CR1000_Table1 20180417.dat",
    "CR1000_Table1 20180419.dat",
    "CR1000_Table1 20180422.dat",
    "CR1000_Table1 20180426.dat",
        ]

START_TIME = '2017-11-08 00:00:00'
END_TIME = '2018-04-26 23:59:50'

DATE_RANGE = pd.date_range(START_TIME, END_TIME, freq='10min')

In [ ]:
COMMON_COLUMNS = ["mLperDrip", "Pyrano_W", "AccdRadSet", "IrrDaily"]

In [ ]:
raw_df = pd.read_csv("./data/" + FILES[0], low_memory=False, skiprows=1, index_col="TIMESTAMP")
raw_df = raw_df.iloc[2:,:]

raw_df.drop(["RECORD", "Batt_Volt"], axis=1)

columns_closed = [columns_to_use for columns_to_use in raw_df.columns if "(3)" in columns_to_use]
columns_open = [columns_to_use for columns_to_use in raw_df.columns if "(4)" in columns_to_use]
columns_closed.extend(COMMON_COLUMNS)
columns_open.extend(COMMON_COLUMNS)


raw_closed = raw_df.loc[:, columns_closed]
raw_open = raw_df.loc[:, columns_open]

In [ ]:
for FILE_NAME in FILES[1:]:
    print(FILE_NAME)
    raw_df = pd.read_csv("./data/" + FILE_NAME, low_memory=False, skiprows=1, index_col="TIMESTAMP")
    raw_df = raw_df.iloc[2:,:]
    
    raw_df.drop(["RECORD", "Batt_Volt"], axis=1)
    
    columns_closed = [columns_to_use for columns_to_use in raw_df.columns if "(3)" in columns_to_use]
    columns_closed.extend(COMMON_COLUMNS)
    if "RootTemp(1)" in raw_df.columns:
        columns_closed.extend(["GrnHsHum(1)", "GrnHsTemp(1)", "RootTemp(1)"])
    
        
    columns_open = [columns_to_use for columns_to_use in raw_df.columns if "(4)" in columns_to_use]
    columns_open.extend(COMMON_COLUMNS)
    if "RootTemp(2)" in raw_df.columns:
        columns_open.extend(["GrnHsHum(2)", "GrnHsTemp(2)", "RootTemp(2)"])
    
    raw_closed = pd.concat([raw_closed, raw_df.loc[:, columns_closed]])
    raw_open = pd.concat([raw_open, raw_df.loc[:, columns_open]])
    print(raw_closed.shape, raw_open.shape)

In [ ]:
raw_open.index = pd.DatetimeIndex(raw_open.index)
raw_closed.index = pd.DatetimeIndex(raw_closed.index)
raw_open = raw_open.astype("float")
raw_closed = raw_closed.astype("float")

DATE_RANGE_OPEN = pd.date_range(START_TIME, END_TIME, freq='10s')
DATE_RANGE_CLOSED = pd.date_range(START_TIME, END_TIME, freq='10s')

raw_open = raw_open.loc[DATE_RANGE_OPEN]
raw_closed = raw_closed.loc[DATE_RANGE_CLOSED]

In [ ]:
raw_open.index = raw_open.index.round(freq="10min")
raw_closed.index = raw_closed.index.round(freq="10min")

raw_open = raw_open.groupby(raw_open.index).mean()
raw_closed = raw_closed.groupby(raw_closed.index).mean()

In [ ]:
# #pH dumping
# raw_closed.loc[["MixTnkpH(4)"]] = raw_open.loc[["MixTnkpH(3)"]]

## Public data (5 min interval)

In [ ]:
TEMPERATURE_FILES = [
    "CR1000_Public 20180112.dat",
    "CR1000_Public 20180213.dat",
    "CR1000_Public 20180216.dat",
    "CR1000_Public 20180219.dat",
        ]

In [ ]:
raw_df = pd.read_csv("./data/" + TEMPERATURE_FILES[0], low_memory=False, skiprows=1, index_col="TIMESTAMP")
raw_df = raw_df.iloc[2:,:]

raw_df.drop(["RECORD", "Batt_Volt"], axis=1)

columns_closed = ["GrnHsTemp(1)"]
columns_open = ["GrnHsTemp(2)"]

temp_closed = raw_df.loc[:, columns_closed]
temp_open = raw_df.loc[:, columns_open]

In [ ]:
for FILE_NAME in TEMPERATURE_FILES[1:]:
    print(FILE_NAME)
    raw_df = pd.read_csv("./data/" + FILE_NAME, low_memory=False, skiprows=1, index_col="TIMESTAMP")
    raw_df = raw_df.iloc[2:,:]
    
    raw_df.drop(["RECORD", "Batt_Volt"], axis=1)
    
    columns_closed = ["GrnHsTemp(1)"]
    columns_open = ["GrnHsTemp(2)"]
    
    temp_closed = pd.concat([temp_closed, raw_df.loc[:, columns_closed]])
    temp_open = pd.concat([temp_open, raw_df.loc[:, columns_open]])
    print(temp_closed.shape, temp_open.shape)

In [ ]:
temp_open.index = pd.DatetimeIndex(temp_open.index)
temp_closed.index = pd.DatetimeIndex(temp_closed.index)
temp_open = temp_open.astype("float")
temp_closed = temp_closed.astype("float")

temp_open.index = temp_open.index.round(freq="5min")
temp_closed.index = temp_closed.index.round(freq="5min")

temp_open = temp_open.groupby(temp_open.index).mean()
temp_closed = temp_closed.groupby(temp_closed.index).mean()

In [ ]:
temp_open.index = temp_open.index.round(freq="10min")
temp_closed.index = temp_closed.index.round(freq="10min")

temp_open = temp_open.groupby(temp_open.index).mean()
temp_closed = temp_closed.groupby(temp_closed.index).mean()

In [ ]:
temp_open = temp_open[temp_open.index < END_TIME]
temp_closed = temp_closed[temp_closed.index < END_TIME]

## Greenhouse data (10 min)

In [ ]:
raw_df = pd.read_excel("./data/Suwon_env_20171102-20180110.xlsx")

In [ ]:
raw_df.index = raw_df["date"].astype("str") + " " + raw_df["time"].astype("str")
raw_df.index = pd.DatetimeIndex(raw_df.index)
raw_df.drop(["date", "time"], axis=1, inplace=True)

In [ ]:
raw_df = raw_df.loc[pd.date_range(START_TIME, temp_open.index[0], freq="10min")]
greenhouse_df = raw_df.loc[:, ["Temp", "Hum"]]

## Manual data

## plant growth data

In [ ]:
plant_growth_df = pd.read_excel("../data/SW_Ion/Plant_grwoth.xlsx", index_col="Date")
plant_growth_df = plant_growth_df.groupby(plant_growth_df.index).mean()
plant_growth_df.index = pd.DatetimeIndex(plant_growth_df.index) + pd.Timedelta("12h")

In [ ]:
plant_growth_df = plant_growth_df.reindex(raw_closed.index)
plant_growth_df.interpolate("linear", inplace=True)

In [ ]:
plant_growth_closed = plant_growth_df[[columns_to_use for columns_to_use in plant_growth_df.columns if "3_" in columns_to_use]]
plant_growth_open = plant_growth_df[[columns_to_use for columns_to_use in plant_growth_df.columns if "4_" in columns_to_use]]

## Merging

### Public data

In [ ]:
raw_closed.loc[temp_closed.index, "GrnHsTemp(1)"] = temp_closed.loc[:, "GrnHsTemp(1)"]
raw_open.loc[temp_open.index, "GrnHsTemp(2)"] = temp_open.loc[:, "GrnHsTemp(2)"]

### Greenhouse data

In [ ]:
raw_closed.loc[greenhouse_df.index, "GrnHsTemp(1)"] = greenhouse_df.loc[:, "Temp"]
raw_open.loc[greenhouse_df.index, "GrnHsTemp(2)"] = greenhouse_df.loc[:, "Temp"]

raw_closed.loc[greenhouse_df.index, "GrnHsHum(1)"] = greenhouse_df.loc[:, "Hum"]
raw_open.loc[greenhouse_df.index, "GrnHsHum(2)"] = greenhouse_df.loc[:, "Hum"]

In [ ]:
CLOSED = ["RWEC(3)", "ECD(3)", "ECS(3)", "DrgV(3)", "RWMC(3)",
          "InjDrgVR(3)", "InjWtrVR(3)", "InjStkVR(3)", "DrgDaily(3)",
          "MixV(3)", "MixTnkpH(3)", "GrnHsHum(1)", "GrnHsTemp(1)", "RootTemp(1)"]
CLOSED.extend(COMMON_COLUMNS)
OPEN = ["RWEC(4)", "ECD(4)", "ECS(4)", "DrgV(4)", "RWMC(4)",
        "InjDrgVR(4)", "InjWtrVR(4)", "InjStkVR(4)", "DrgDaily(4)",
        "MixV(4)", "MixTnkpH(4)", "GrnHsHum(2)", "GrnHsTemp(2)", "RootTemp(2)"]
OPEN.extend(COMMON_COLUMNS)

In [ ]:
raw_closed = raw_closed.loc[:, CLOSED]
raw_open = raw_open.loc[:, OPEN]

## plant growth data

In [ ]:
raw_closed = pd.concat([raw_closed, plant_growth_closed], axis=1)
raw_open = pd.concat([raw_open, plant_growth_open], axis=1)

In [ ]:
raw_closed.head()

In [ ]:
RESULTS_DIRECTORY = "./results/"

if not os.path.exists(RESULTS_DIRECTORY):
    os.makedirs(RESULTS_DIRECTORY)

In [ ]:
raw_closed.to_csv(RESULTS_DIRECTORY + "closed_data.csv")
raw_open.to_csv(RESULTS_DIRECTORY + "open_data.csv")